In [7]:
from flask import Flask, render_template, request, redirect, url_for,flash
from flask_mysqldb import MySQL
import os
import mysql.connector
import pandas as pd
from flask import *
import urllib3
import requests
import array as ar
import random
import smtplib
import csv as cs

app = Flask(__name__)


app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'exam'

mysql = MySQL(app)
@app.route('/', methods=['GET', 'POST'])
def dashboard():
    
    
    return render_template('admin.html')

@app.route('/student', methods=['GET', 'POST'])
def admin_stu():
    
    if request.method =="POST":
        
        cur = mysql.connection.cursor()
        details = request.form
        sbranch = details['sbranch']
        ssem = details['sem']
        sql = "SELECT * from student where sbranch=%s and sem=%s"
        data = (sbranch,ssem,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        #print(myresult[0][0])
        if(cur.rowcount==0):
            flash('Student not found')
        else:
            result = myresult
            return render_template('admin_stu.html',data=result)
        cur.close()
    return render_template('admin_stu.html')

@app.route('/addstudentmanually', methods=['GET', 'POST'])
def add_stu_manually():
    
    if request.method =="POST":
        MAX_LEN = 12
        DIGITS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 
        LOCASE_CHARACTERS = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
                     'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q',
                     'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
                     'z']
 
        UPCASE_CHARACTERS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                     'I', 'J', 'K', 'M', 'N', 'O', 'p', 'Q',
                     'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
                     'Z']
 
        SYMBOLS = ['@', '#', '$', '%', '=', ':', '?', '.', '/', '|', '~', '>',
                   '*', '(', ')', '<']
        COMBINED_LIST = DIGITS + UPCASE_CHARACTERS + LOCASE_CHARACTERS + SYMBOLS
        
        
        rand_digit = random.choice(DIGITS)
        rand_upper = random.choice(UPCASE_CHARACTERS)
        rand_lower = random.choice(LOCASE_CHARACTERS)
        rand_symbol = random.choice(SYMBOLS)
        temp_pass = rand_digit + rand_upper + rand_lower + rand_symbol
        for x in range(MAX_LEN - 4):
            temp_pass = temp_pass + random.choice(COMBINED_LIST)
            temp_pass_list = ar.array('u', temp_pass)
            random.shuffle(temp_pass_list)
        password = ""
        for x in temp_pass_list:
            password = password + x
        
        gmail_user = 'temporarily97@gmail.com'
        gmail_password = 'tiajxsgmlosgyxyu'
        
        sent_from = gmail_user
        
        print("COMES HERE 90")
        cur = mysql.connection.cursor()
        details = request.form
        sid = details['sid']
        sname = details['sname']
        sbranch = details['sbranch']
        ssem = details['sem']
        semail = details['email']
        spwd = password
        to = semail
        email = []
        email.append(semail)
        sql = "SELECT * from student where sid=%s"
        data = (sid,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        #print(myresult[0][0])
        if(cur.rowcount==1):
            flash('Student already there')
            cur.close()
            return render_template('s_signup.html')
        else:
            cur = mysql.connection.cursor()
            sql1 = 'INSERT INTO student VALUES (%s,%s,%s,%s,%s,%s)'
            data1 = (sid,spwd,sname,sbranch,ssem,semail,)
            cur.execute(sql1,data1)
            mysql.connection.commit()
            myresult1 = cur.fetchall()
            if(cur.rowcount==1):
                server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
                server.ehlo()
                print("COMES HERE 121")
                server.login(gmail_user, gmail_password)
                print("COMES HERE rowcount 123")
                for i in range(len(email)):
                    subject = 'Login ID and password for Examination system.'
                    body ='Hello Student Welcome to examination system this mail contains your Login Id and Password.' + '\nID:' + ' ' + sid + '\n' + 'Password:'+ ' ' + spwd

                    email_text = """\
                    From: %s
                    To: %s
                    Subject: %s

                    %s
                    """ % (sent_from, ", ".join(to), subject, body)

                    try:

                        server.sendmail(sent_from,email[i],email_text)

                        print ('Email sent!')
                    except:
                        print ('Something went wrong...')

                server.close()
                flash("Student Added Successfully")
                return render_template('s_signup.html')
            else:
                flash("Error Something Went Wrong")
            return render_template('s_signup.html')
        cur.close()
    return render_template('s_signup.html')

@app.route('/teacher', methods=['GET', 'POST'])
def admin_tea():
    
    if request.method =="POST":
        
        cur = mysql.connection.cursor()
        details = request.form
        tbranch = details['tbranch']
        #ssem = details['sem']
        sql = "SELECT * from teacher where branch=%s"
        data = (tbranch,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        #print(myresult[0][0])
        if(cur.rowcount==0):
            flash('Teacher not found')
        else:
            result = myresult
            return render_template('admin_tea.html',data=result)
        cur.close()
    return render_template('admin_tea.html')

@app.route('/addteachermanually', methods=['GET', 'POST'])
def add_tea_manually():
    
    if request.method =="POST":
        MAX_LEN = 12
        DIGITS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 
        LOCASE_CHARACTERS = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
                     'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q',
                     'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
                     'z']
 
        UPCASE_CHARACTERS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                     'I', 'J', 'K', 'M', 'N', 'O', 'p', 'Q',
                     'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
                     'Z']
 
        SYMBOLS = ['@', '#', '$', '%', '=', ':', '?', '.', '/', '|', '~', '>',
                   '*', '(', ')', '<']
        COMBINED_LIST = DIGITS + UPCASE_CHARACTERS + LOCASE_CHARACTERS + SYMBOLS
        
        
        rand_digit = random.choice(DIGITS)
        rand_upper = random.choice(UPCASE_CHARACTERS)
        rand_lower = random.choice(LOCASE_CHARACTERS)
        rand_symbol = random.choice(SYMBOLS)
        temp_pass = rand_digit + rand_upper + rand_lower + rand_symbol
        for x in range(MAX_LEN - 4):
            temp_pass = temp_pass + random.choice(COMBINED_LIST)
            temp_pass_list = ar.array('u', temp_pass)
            random.shuffle(temp_pass_list)
        password = ""
        for x in temp_pass_list:
            password = password + x
        
        gmail_user = 'temporarily97@gmail.com'
        gmail_password = 'tiajxsgmlosgyxyu'
        
        sent_from = gmail_user
        
        
        cur = mysql.connection.cursor()
        details = request.form
        tid = details['tid']
        tname = details['tname']
        tbranch = details['tbranch']
        temail = details['email']
        tpwd = password
        to = temail
        email = []
        email.append(temail)
        sql = "SELECT * from teacher where tid=%s"
        data = (tid,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        #print(myresult[0][0])
        if(cur.rowcount==1):
            flash('Teacher already there')
            cur.close()
            return render_template('t_signup.html')
        else:
            cur = mysql.connection.cursor()
            sql1 = 'INSERT INTO teacher VALUES (%s,%s,%s,%s,%s)'
            data1 = (tid,tpwd,tname,tbranch,temail,)
            cur.execute(sql1,data1)
            mysql.connection.commit()
            myresult1 = cur.fetchall()
            if(cur.rowcount==1):
                server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
                server.ehlo()
                server.login(gmail_user, gmail_password)
                for i in range(len(email)):
                    subject = 'Login ID and password for Examination system.'
                    body ='Hello Teacher Welcome to examination system this mail contains your Login Id and Password.' + '\nID:' + ' ' + tid + '\n' + 'Password:'+ ' ' + tpwd

                    email_text = """\
                    From: %s
                    To: %s
                    Subject: %s

                    %s
                    """ % (sent_from, ", ".join(to), subject, body)

                    try:

                        server.sendmail(sent_from,email[i],email_text)

                        print ('Email sent!')
                    except:
                        print ('Something went wrong...')

                server.close()
                flash("Teacher Added Successfully")
                return render_template('t_signup.html')
            else:
                flash("Error Something Went Wrong")
            return render_template('t_signup.html')
        cur.close()
    return render_template('t_signup.html')
  
          
@app.route('/remove_student', methods=['GET', 'POST'])
def remove_student():
    
     if request.method == "POST":
            
        cur = mysql.connection.cursor()
       
        details = request.form
        sid = int(details['sid'])
        sql = "DELETE from student where sid=%s"
        data = (sid,)
        cur.execute(sql,data)
        mysql.connection.commit()
        myresult = cur.fetchall()
        cur.close()
        if(cur.rowcount==1):
            flash('Successfully deleted the student')
            return redirect(url_for('admin_stu'))
        else:
            flash('Error')
        return redirect(url_for('admin_stu'))
    
@app.route('/remove_teacher', methods=['GET', 'POST'])
def remove_teacher():
    
     if request.method == "POST":
            
        cur = mysql.connection.cursor()
       
        details = request.form
        tid = int(details['tid'])
        sql = "DELETE from teacher where tid=%s"
        data = (tid,)
        cur.execute(sql,data)
        mysql.connection.commit()
        myresult = cur.fetchall()
        cur.close()
        if(cur.rowcount==1):
            flash('Successfully deleted the teacher')
            return redirect(url_for('admin_tea'))
        else:
            flash('Error')
        return redirect(url_for('admin_tea'))
    
@app.route('/uploadcsv', methods=['GET', 'POST'])
def upload_csv():
   
    if request.method == 'POST':
        details = request.form
        #random password generation and storing of client side csv file
        MAX_LEN = 12
        DIGITS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 
        LOCASE_CHARACTERS = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
                     'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q',
                     'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
                     'z']
 
        UPCASE_CHARACTERS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                     'I', 'J', 'K', 'M', 'N', 'O', 'p', 'Q',
                     'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
                     'Z']
 
        SYMBOLS = ['@', '#', '$', '%', '=', ':', '?', '.', '/', '|', '~', '>',
                   '*', '(', ')', '<']
        COMBINED_LIST = DIGITS + UPCASE_CHARACTERS + LOCASE_CHARACTERS + SYMBOLS
        
        passlist = []
        
        if (details['submit']=='Upload Students'):
            
            #print("s upload")
            csv1  = request.files['sfile']
            
            data1 = pd.read_csv(csv1)
            for i,row in data1.iterrows():
                
                rand_digit = random.choice(DIGITS)
                rand_upper = random.choice(UPCASE_CHARACTERS)
                rand_lower = random.choice(LOCASE_CHARACTERS)
                rand_symbol = random.choice(SYMBOLS)
                temp_pass = rand_digit + rand_upper + rand_lower + rand_symbol
                for x in range(MAX_LEN - 4):
                    temp_pass = temp_pass + random.choice(COMBINED_LIST)
                    temp_pass_list = ar.array('u', temp_pass)
                    random.shuffle(temp_pass_list)
                password = ""
                for x in temp_pass_list:
                    password = password + x
                passlist.append(password)
                
            #csv_input = pd.read_csv(csv)        
            x = pd.Series(passlist)
            data1['Spwd'] = x
            with open('Recently_Uploaded_students.csv', 'w') as f:
                data1.to_csv(f,index=False)
            
            df = pd.read_csv('Recently_Uploaded_students.csv')
            df_reorder = data1[['SID','Spwd','Sname','Sbranch','Sem','Email']] # rearrange column here
            df_reorder.to_csv('Recently_Uploaded_students.csv',index=False)

            cur = mysql.connection.cursor()
            data = pd.read_csv('Recently_Uploaded_students.csv')

            for i,row in data.iterrows():
                sql = "INSERT INTO student VALUES (%s,%s,%s,%s,%s,%s)"
                cur.execute(sql,tuple(row))
                mysql.connection.commit()
            if(cur.rowcount>=1):
                flash('Successfully Uploaded the data')
            else:
                flash('Error Check CSV file or data is already there')
    
            cur.close()
            #sending password through email code
            gmail_user = 'temporarily97@gmail.com'
            gmail_password = 'tiajxsgmlosgyxyu'
            filename = open('Recently_Uploaded_students.csv', 'r')

            file = cs.DictReader(filename)
            email = []
            
            passwords = []
            login_ID = []
            for col in file:
                email.append(col['Email'])
                passwords.append(col['Spwd'])
                login_ID.append(col['SID'])
            sent_from = gmail_user
            to = email
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            
            for i in range(len(email)):
                
                subject = 'Login ID and password for Examination system.'
                body ='Hello Student Welcome to examination system this mail contains your Login Id and Password.' + '\nID:' + ' ' + login_ID[i] + '\n' + 'Password:'+ ' ' + passwords[i]

                email_text = """\
                From: %s
                To: %s
                Subject: %s

                %s
                """ % (sent_from, ", ".join(to), subject, body)

                try:

                    server.sendmail(sent_from,email[i], email_text)

                    print ('Email sent!')
                except:
                    print ('Something went wrong...')

            server.close()
            return render_template('Upload_csv.html')

        if(details['submit']=='Upload Teachers'):
        
            #print("t upload")
            csv1  = request.files['tfile']
            data1 = pd.read_csv(csv1)
            for i,row in data1.iterrows():
                
                rand_digit = random.choice(DIGITS)
                rand_upper = random.choice(UPCASE_CHARACTERS)
                rand_lower = random.choice(LOCASE_CHARACTERS)
                rand_symbol = random.choice(SYMBOLS)
                temp_pass = rand_digit + rand_upper + rand_lower + rand_symbol
                for x in range(MAX_LEN - 4):
                    temp_pass = temp_pass + random.choice(COMBINED_LIST)
                    temp_pass_list = ar.array('u', temp_pass)
                    random.shuffle(temp_pass_list)
                password = ""
                for x in temp_pass_list:
                    password = password + x
                passlist.append(password)
            
            #data = pd.read_csv(csv)
            #array = data.to_numpy()
            x = pd.Series(passlist)
            data1['Tpwd'] = x
            with open('Recently_Uploaded_teachers.csv', 'w') as f:
                data1.to_csv(f,index=False)
                
            df = pd.read_csv('Recently_Uploaded_teachers.csv')
            df_reorder = data1[['TID','Tpwd','TName','Branch','Email']] # rearrange column here
            df_reorder.to_csv('Recently_Uploaded_teachers.csv',index=False)
            
            cur = mysql.connection.cursor()
            data = pd.read_csv('Recently_Uploaded_teachers.csv')

            for i,row in data.iterrows():

                sql = "INSERT INTO teacher VALUES (%s,%s,%s,%s,%s)"
                cur.execute(sql, tuple(row))
                mysql.connection.commit()
            if(cur.rowcount>=1):
                flash('Successfully Uploaded the data')
            else:
                flash('Error Check CSV file or data is already there')
            cur.close()
            
            #sending password through email code
            gmail_user = 'temporarily97@gmail.com'
            gmail_password = 'tiajxsgmlosgyxyu'
            filename = open('Recently_Uploaded_teachers.csv', 'r')
            file = cs.DictReader(filename)
            email = []
            passwords = []
            login_ID = []
            for col in file:
                email.append(col['Email'])
                passwords.append(col['Tpwd'])
                login_ID.append(col['TID'])
            sent_from = gmail_user
            to = email
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            for i in range(len(email)):
                
                subject = 'Login ID and password for Examination system.'
                body ='Hello Teacher Welcome to examination system this mail contains your Login ID and password.' + '\n' + 'ID:' + ' ' + login_ID[i] + '\n' + 'Password:'+ ' ' + passwords[i]

                email_text = """\
                From: %s
                To: %s
                Subject: %s

                %s
                """ % (sent_from, ", ".join(to), subject, body)

                try:
                    
                    server.sendmail(sent_from,email[i], email_text)

                    print ('Email sent!')
                except:
                    print ('Something went wrong...')

            server.close()
            
            
            return render_template('Upload_csv.html')
        
        if (details['submit']=='Upload Courses'):
            
            #print("c upload")
            csv  = request.files['cfile']
            data = pd.read_csv(csv)
            array = data.to_numpy()
            cur = mysql.connection.cursor()
            for i,row in data.iterrows():

                sql = "INSERT INTO course VALUES (%s,%s,%s,%s)"
                cur.execute(sql, tuple(row))
                mysql.connection.commit()
            if(cur.rowcount>=1):
                flash('Successfully Uploaded the data')
            else:
                flash('Error Check CSV file or data is already there')
    
            cur.close()
            return render_template('Upload_csv.html')

    return render_template('Upload_csv.html')

if __name__ == '__main__':
    app.secret_key = os.urandom(24)
    app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2021 17:38:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2021 17:38:35] "GET /student HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2021 17:38:36] "GET /addstudentmanually HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2021 17:38:36] "GET /static/img_avatar2.png HTTP/1.1" 404 -


COMES HERE 90
COMES HERE 121
COMES HERE rowcount 123


127.0.0.1 - - [18/May/2021 17:38:55] "POST /addstudentmanually HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2021 17:38:56] "GET /static/img_avatar2.png HTTP/1.1" 404 -


Email sent!
